In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# Load dataset
file_path = 'IMPACT.sensors.csv'
data = pd.read_csv(file_path)

# Ensure timestamp exists
timestamp_column = 'createdAt'
data[timestamp_column] = pd.to_datetime(data[timestamp_column])

# Sort data by time
data = data.sort_values(by=timestamp_column)

# Select features and target (Oxygen)
features = ['oxygen']
data_features = data[features]

# Normalize data
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data_features)

# Define sequence length and prediction steps
sequence_length = 12  # Using past 10 steps (100 min)
prediction_steps = [6, 12, 18, 24, 30, 36, 42, 48]  # Predict 1-8 hours ahead (10 min intervals)

# Prepare sequences and targets
def create_future_sequences(data, sequence_length, prediction_steps):
    X, y = [], []
    for i in range(len(data) - sequence_length - max(prediction_steps)):
        X.append(data[i:i + sequence_length])
        future_values = [np.mean(data[i + sequence_length + p: i + sequence_length + p + 6]) for p in prediction_steps]
        y.append(np.array(future_values).flatten())
    return np.array(X), np.array(y)

X, y = create_future_sequences(data_normalized, sequence_length, prediction_steps)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build GRU Model
input_layer = Input(shape=(sequence_length, len(features)))
x = GRU(64, activation='relu', return_sequences=True)(input_layer)
x = GRU(32, activation='relu', return_sequences=False)(x)
dense1 = Dense(64, activation='relu')(x)
output_layer = Dense(len(prediction_steps), activation='linear')(dense1)

# Build and compile model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=42, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss = model.evaluate(X_test, y_test)
print("Test Loss (MSE):", test_loss)

# Calculate additional evaluation metrics
y_pred = model.predict(X_test)

def evaluate_predictions(y_true, y_pred, prediction_steps):
    print("\nStep-wise Performance Evaluation:")
    for idx, step in enumerate(prediction_steps):
        hours = (step * 10) // 60
        minutes = (step * 10) % 60
        y_true_step = y_true[:, idx]
        y_pred_step = y_pred[:, idx]

        r2 = r2_score(y_true_step, y_pred_step)
        mae = mean_absolute_error(y_true_step, y_pred_step)
        print(f"{hours}h{minutes:02d}min: R²={r2:.2f}, MAE={mae:.2f}")

    overall_r2 = r2_score(y_test, y_pred)
    overall_mae = mean_absolute_error(y_test, y_pred)
    print("\nOverall Performance:")
    print(f"Overall R² Score: {overall_r2:.2f}")
    print(f"Overall Mean Absolute Error (MAE): {overall_mae:.2f}")

evaluate_predictions(y_test, y_pred, prediction_steps)

# Save the model
model.save('oxygen_prediction_gru_model.keras')

Epoch 1/42
491/491 [==============================] - 7s 7ms/step - loss: 0.0196 - val_loss: 0.0066
Epoch 2/42
491/491 [==============================] - 3s 6ms/step - loss: 0.0065 - val_loss: 0.0068
Epoch 3/42
491/491 [==============================] - 3s 6ms/step - loss: 0.0065 - val_loss: 0.0065
Epoch 4/42
491/491 [==============================] - 3s 6ms/step - loss: 0.0064 - val_loss: 0.0064
Epoch 5/42
491/491 [==============================] - 3s 6ms/step - loss: 0.0064 - val_loss: 0.0066
Epoch 6/42
491/491 [==============================] - 3s 6ms/step - loss: 0.0062 - val_loss: 0.0066
Epoch 7/42
491/491 [==============================] - 3s 6ms/step - loss: 0.0063 - val_loss: 0.0064
Epoch 8/42
491/491 [==============================] - 3s 6ms/step - loss: 0.0062 - val_loss: 0.0065
Epoch 9/42
491/491 [==============================] - 3s 6ms/step - loss: 0.0061 - val_loss: 0.0063
Epoch 10/42
491/491 [==============================] - 3s 6ms/step - loss: 0.0061 - val_loss: 0.0062